# QUBO Fundamentals: Converting Problems to Quantum Form

## Learning Objectives
- Understand QUBO (Quadratic Unconstrained Binary Optimization) formulation
- Learn to convert constrained problems to QUBO form
- Master penalty methods for constraint handling
- Practice with real optimization problems

## What is QUBO?

QUBO problems have the form:
$$\min_{x \in \{0,1\}^n} x^T Q x$$

Where:
- $x$ is a binary vector
- $Q$ is the QUBO matrix (upper triangular)
- Linear terms appear on the diagonal
- Quadratic terms appear off-diagonal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
import itertools
from dataclasses import dataclass

# Set random seed for reproducibility
np.random.seed(42)

print("QUBO Tutorial Environment Ready!")

## 1. Basic QUBO Representation

Let's start with a simple example to understand QUBO structure.

In [ ]:
@dataclass
class QUBOProblem:
    """Represents a QUBO optimization problem"""
    Q: np.ndarray  # QUBO matrix
    variable_names: List[str]  # Names of binary variables
    
    def evaluate(self, x: np.ndarray) -> float:
        """Evaluate QUBO objective for binary vector x"""
        return x.T @ self.Q @ x
    
    def solve_brute_force(self) -> Tuple[np.ndarray, float]:
        """Solve small QUBO problems by exhaustive search"""
        n = len(self.variable_names)
        best_x = None
        best_obj = float('inf')
        
        # Try all 2^n possible binary assignments
        for bits in itertools.product([0, 1], repeat=n):
            x = np.array(bits)
            obj = self.evaluate(x)
            if obj < best_obj:
                best_obj = obj
                best_x = x.copy()
        
        return best_x, best_obj
    
    def print_solution(self, x: np.ndarray, obj: float):
        """Pretty print solution"""
        print(f"Optimal objective: {obj}")
        print("Variable assignments:")
        for i, (var, val) in enumerate(zip(self.variable_names, x)):
            print(f"  {var} = {val}")

# Example: Simple 2-variable QUBO
# Minimize: x1 - 2*x2 + 3*x1*x2
Q_simple = np.array([
    [1.0, 1.5],   # x1 coefficient on diagonal, half of x1*x2 coefficient off-diagonal
    [1.5, -2.0]   # x2 coefficient on diagonal
])

simple_qubo = QUBOProblem(Q_simple, ['x1', 'x2'])

print("Simple QUBO Matrix:")
print(Q_simple)
print("\nObjective: x1 - 2*x2 + 3*x1*x2")

# Test all combinations
print("\nEvaluating all combinations:")
for x1, x2 in [(0,0), (0,1), (1,0), (1,1)]:
    x = np.array([x1, x2])
    obj = simple_qubo.evaluate(x)
    print(f"x=({x1},{x2}): objective = {obj}")

# Find optimal solution
opt_x, opt_obj = simple_qubo.solve_brute_force()
print("\nOptimal solution:")
simple_qubo.print_solution(opt_x, opt_obj)

## 2. Converting Linear Constraints to QUBO

Real optimization problems have constraints. We convert them using penalty methods:

$$\min f(x) \text{ subject to } g(x) = 0$$

becomes:

$$\min f(x) + P \cdot g(x)^2$$

where $P$ is a penalty parameter.

In [ ]:
class QUBOConverter:
    """Converts constrained problems to QUBO form"""
    
    def __init__(self):
        self.variables = []
        self.objective_terms = {}  # (i,j) -> coefficient
        self.constraints = []
        
    def add_variable(self, name: str) -> int:
        """Add binary variable, return its index"""
        if name not in self.variables:
            self.variables.append(name)
        return self.variables.index(name)
    
    def add_linear_term(self, var_idx: int, coeff: float):
        """Add linear term to objective"""
        key = (var_idx, var_idx)
        self.objective_terms[key] = self.objective_terms.get(key, 0) + coeff
    
    def add_quadratic_term(self, var1_idx: int, var2_idx: int, coeff: float):
        """Add quadratic term to objective"""
        # Ensure upper triangular
        if var1_idx > var2_idx:
            var1_idx, var2_idx = var2_idx, var1_idx
        
        key = (var1_idx, var2_idx)
        self.objective_terms[key] = self.objective_terms.get(key, 0) + coeff
    
    def add_equality_constraint(self, var_coeffs: Dict[int, float], 
                              constant: float, penalty: float):
        """Add equality constraint with penalty method"""
        # Constraint: sum(coeff[i] * x[i]) = constant
        # Penalty: penalty * (sum(coeff[i] * x[i]) - constant)^2
        
        # Expand: penalty * (sum - const)^2 = 
        # penalty * (sum^2 - 2*const*sum + const^2)
        
        # Add quadratic terms
        for i, coeff_i in var_coeffs.items():
            for j, coeff_j in var_coeffs.items():
                if i <= j:  # Upper triangular
                    quad_coeff = penalty * coeff_i * coeff_j
                    if i == j:
                        self.add_linear_term(i, quad_coeff)
                    else:
                        self.add_quadratic_term(i, j, quad_coeff)
        
        # Add linear terms from -2*const*sum
        for i, coeff_i in var_coeffs.items():
            self.add_linear_term(i, -2 * penalty * constant * coeff_i)
        
        # Constant term (ignored in optimization)
        # penalty * constant^2
    
    def build_qubo_matrix(self) -> np.ndarray:
        """Build QUBO matrix from accumulated terms"""
        n = len(self.variables)
        Q = np.zeros((n, n))
        
        for (i, j), coeff in self.objective_terms.items():
            Q[i, j] = coeff
        
        return Q
    
    def get_qubo_problem(self) -> QUBOProblem:
        """Get final QUBO problem"""
        Q = self.build_qubo_matrix()
        return QUBOProblem(Q, self.variables.copy())

print("QUBO Converter class ready!")

## 3. Example: Knapsack Problem to QUBO

The 0-1 Knapsack Problem:
- **Objective**: Maximize value of selected items
- **Constraint**: Total weight ≤ capacity

**Mathematical formulation**:
$$\max \sum_{i=1}^n v_i x_i \text{ subject to } \sum_{i=1}^n w_i x_i \leq W$$

**QUBO conversion steps**:
1. Convert maximization to minimization: multiply by -1
2. Convert inequality to equality using slack variables
3. Apply penalty method for constraint

In [ ]:
def knapsack_to_qubo(values: List[float], weights: List[float], 
                     capacity: float, penalty: float = 10.0) -> QUBOProblem:
    """Convert knapsack problem to QUBO form"""
    
    n_items = len(values)
    
    # We need slack variables to convert inequality to equality
    # sum(w_i * x_i) + slack = capacity
    # Slack is binary: we need log2(capacity) slack variables
    n_slack = int(np.ceil(np.log2(capacity + 1)))
    
    converter = QUBOConverter()
    
    # Add item variables
    item_vars = []
    for i in range(n_items):
        var_idx = converter.add_variable(f'item_{i}')
        item_vars.append(var_idx)
        # Add negative value (since we minimize)
        converter.add_linear_term(var_idx, -values[i])
    
    # Add slack variables
    slack_vars = []
    for i in range(n_slack):
        var_idx = converter.add_variable(f'slack_{i}')
        slack_vars.append(var_idx)
    
    # Build constraint: sum(w_i * x_i) + sum(2^j * s_j) = capacity
    constraint_coeffs = {}
    
    # Item weights
    for i, weight in enumerate(weights):
        constraint_coeffs[item_vars[i]] = weight
    
    # Slack weights (powers of 2)
    for i, var_idx in enumerate(slack_vars):
        constraint_coeffs[var_idx] = 2**i
    
    # Add equality constraint
    converter.add_equality_constraint(constraint_coeffs, capacity, penalty)
    
    return converter.get_qubo_problem()

# Example: Small knapsack problem
values = [10, 40, 30, 50]  # Item values
weights = [5, 4, 6, 3]     # Item weights
capacity = 10              # Knapsack capacity

print("Knapsack Problem:")
print(f"Items: {list(zip(values, weights))} (value, weight)")
print(f"Capacity: {capacity}")

# Convert to QUBO
knapsack_qubo = knapsack_to_qubo(values, weights, capacity, penalty=100)

print(f"\nQUBO has {len(knapsack_qubo.variable_names)} variables:")
print(knapsack_qubo.variable_names)

print("\nQUBO Matrix:")
print(knapsack_qubo.Q)

# Solve (if small enough)
if len(knapsack_qubo.variable_names) <= 10:
    opt_x, opt_obj = knapsack_qubo.solve_brute_force()
    print("\nOptimal QUBO solution:")
    knapsack_qubo.print_solution(opt_x, opt_obj)
    
    # Interpret solution
    n_items = len(values)
    selected_items = []
    total_value = 0
    total_weight = 0
    
    for i in range(n_items):
        if opt_x[i] == 1:
            selected_items.append(i)
            total_value += values[i]
            total_weight += weights[i]
    
    print(f"\nKnapsack interpretation:")
    print(f"Selected items: {selected_items}")
    print(f"Total value: {total_value}")
    print(f"Total weight: {total_weight}/{capacity}")
else:
    print("Problem too large for brute force solution")

## 4. Example: Number Partitioning Problem

Given a set of numbers, partition them into two subsets with equal sums.

**Problem**: Partition numbers $\{a_1, a_2, \ldots, a_n\}$ into sets $S_1, S_2$ such that:
$$\sum_{i \in S_1} a_i = \sum_{i \in S_2} a_i$$

**QUBO formulation**: Minimize the squared difference between subset sums:
$$\min \left(\sum_{i=1}^n a_i x_i - \sum_{i=1}^n a_i (1-x_i)\right)^2$$

where $x_i = 1$ if number $a_i$ is in subset $S_1$, $0$ if in $S_2$.

In [ ]:
def number_partition_to_qubo(numbers: List[float]) -> QUBOProblem:
    """Convert number partitioning to QUBO"""
    
    converter = QUBOConverter()
    
    # Add variables for each number
    var_indices = []
    for i, num in enumerate(numbers):
        var_idx = converter.add_variable(f'x_{i}')
        var_indices.append(var_idx)
    
    # Objective: minimize (sum1 - sum2)^2
    # sum1 = sum(a_i * x_i)
    # sum2 = sum(a_i * (1 - x_i)) = sum(a_i) - sum(a_i * x_i)
    # Difference = sum(a_i * x_i) - (sum(a_i) - sum(a_i * x_i))
    #            = 2 * sum(a_i * x_i) - sum(a_i)
    
    total_sum = sum(numbers)
    
    # We want to minimize (2 * sum(a_i * x_i) - total_sum)^2
    # Expanding: 4 * (sum(a_i * x_i))^2 - 4 * total_sum * sum(a_i * x_i) + total_sum^2
    
    # Quadratic terms: 4 * sum_i sum_j (a_i * a_j * x_i * x_j)
    for i, a_i in enumerate(numbers):
        for j, a_j in enumerate(numbers):
            if i <= j:
                coeff = 4 * a_i * a_j
                if i == j:
                    converter.add_linear_term(var_indices[i], coeff)
                else:
                    converter.add_quadratic_term(var_indices[i], var_indices[j], coeff)
    
    # Linear terms: -4 * total_sum * sum(a_i * x_i)
    for i, a_i in enumerate(numbers):
        converter.add_linear_term(var_indices[i], -4 * total_sum * a_i)
    
    # Constant term (total_sum^2) is ignored in optimization
    
    return converter.get_qubo_problem()

# Example: Partition numbers
numbers = [3, 1, 1, 2, 2, 1]
print(f"Numbers to partition: {numbers}")
print(f"Total sum: {sum(numbers)}")
print(f"Target sum per subset: {sum(numbers)/2}")

# Convert to QUBO
partition_qubo = number_partition_to_qubo(numbers)

print(f"\nQUBO Matrix:")
print(partition_qubo.Q)

# Solve
opt_x, opt_obj = partition_qubo.solve_brute_force()
print("\nOptimal solution:")
partition_qubo.print_solution(opt_x, opt_obj)

# Interpret solution
subset1 = [numbers[i] for i in range(len(numbers)) if opt_x[i] == 1]
subset2 = [numbers[i] for i in range(len(numbers)) if opt_x[i] == 0]

print(f"\nPartition interpretation:")
print(f"Subset 1: {subset1}, sum = {sum(subset1)}")
print(f"Subset 2: {subset2}, sum = {sum(subset2)}")
print(f"Difference: {abs(sum(subset1) - sum(subset2))}")

# Verify it's a valid partition
assert set(subset1 + subset2) == set(numbers), "Invalid partition!"
print("✓ Valid partition confirmed")

## 5. Challenge Exercises

### Exercise 1: Maximum Independent Set
Convert the Maximum Independent Set problem to QUBO form.

**Problem**: Given a graph, find the largest set of vertices with no edges between them.

**Hint**: 
- Maximize the number of selected vertices
- Add penalty for selecting adjacent vertices

### Exercise 2: Graph Coloring
Convert 3-coloring problem to QUBO form.

**Problem**: Color graph vertices with 3 colors such that no adjacent vertices have the same color.

**Hint**:
- Use 3 binary variables per vertex (one per color)
- Each vertex must have exactly one color
- Adjacent vertices cannot share colors

### Exercise 3: Traveling Salesman Problem (TSP)
Convert a small TSP instance to QUBO form.

**Problem**: Find shortest route visiting all cities exactly once.

**Hint**:
- Use binary variables $x_{i,j}$ = 1 if city $i$ is visited at position $j$
- Each city visited exactly once
- Each position has exactly one city
- Add subtour elimination constraints

In [ ]:
# SOLUTIONS TO EXERCISES

# Exercise 1 Solution: Maximum Independent Set
def max_independent_set_to_qubo(adj_matrix: np.ndarray, penalty: float = 10.0) -> QUBOProblem:
    """Convert Maximum Independent Set to QUBO"""
    n = adj_matrix.shape[0]
    converter = QUBOConverter()
    
    # Add vertex variables
    vertex_vars = []
    for i in range(n):
        var_idx = converter.add_variable(f'v_{i}')
        vertex_vars.append(var_idx)
        # Maximize vertices selected (minimize negative)
        converter.add_linear_term(var_idx, -1.0)
    
    # Add penalty for adjacent vertices both selected
    for i in range(n):
        for j in range(i+1, n):
            if adj_matrix[i, j] == 1:  # Edge exists
                converter.add_quadratic_term(vertex_vars[i], vertex_vars[j], penalty)
    
    return converter.get_qubo_problem()

# Test Maximum Independent Set
print("=== Exercise 1: Maximum Independent Set ===")
# Simple graph: 0-1-2-3 (path)
adj_matrix = np.array([
    [0, 1, 0, 0],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 0]
])

print("Graph adjacency matrix:")
print(adj_matrix)

mis_qubo = max_independent_set_to_qubo(adj_matrix)
opt_x, opt_obj = mis_qubo.solve_brute_force()

print("\nOptimal solution:")
mis_qubo.print_solution(opt_x, opt_obj)

selected_vertices = [i for i in range(len(opt_x)) if opt_x[i] == 1]
print(f"Selected vertices: {selected_vertices}")
print(f"Independent set size: {len(selected_vertices)}")

# Verify independence
for i in selected_vertices:
    for j in selected_vertices:
        if i != j and adj_matrix[i, j] == 1:
            print(f"ERROR: Vertices {i} and {j} are adjacent!")
            break
else:
    print("✓ Valid independent set confirmed")

In [ ]:
# Exercise 2 Solution: Graph 3-Coloring
def graph_3coloring_to_qubo(adj_matrix: np.ndarray, penalty: float = 10.0) -> QUBOProblem:
    """Convert 3-coloring problem to QUBO"""
    n = adj_matrix.shape[0]
    converter = QUBOConverter()
    
    # Variables: x_{i,c} = 1 if vertex i has color c
    vertex_color_vars = {}
    for i in range(n):
        for c in range(3):
            var_idx = converter.add_variable(f'v{i}_c{c}')
            vertex_color_vars[(i, c)] = var_idx
    
    # Constraint 1: Each vertex has exactly one color
    for i in range(n):
        # sum_c x_{i,c} = 1
        color_vars = {vertex_color_vars[(i, c)]: 1.0 for c in range(3)}
        converter.add_equality_constraint(color_vars, 1.0, penalty)
    
    # Constraint 2: Adjacent vertices have different colors
    for i in range(n):
        for j in range(i+1, n):
            if adj_matrix[i, j] == 1:  # Edge exists
                # For each color c: x_{i,c} + x_{j,c} <= 1
                # Equivalently: x_{i,c} * x_{j,c} = 0 (penalty term)
                for c in range(3):
                    converter.add_quadratic_term(
                        vertex_color_vars[(i, c)], 
                        vertex_color_vars[(j, c)], 
                        penalty
                    )
    
    return converter.get_qubo_problem()

# Test 3-coloring on triangle graph
print("\n=== Exercise 2: Graph 3-Coloring ===")
# Triangle graph (requires 3 colors)
triangle_adj = np.array([
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0]
])

print("Triangle graph adjacency matrix:")
print(triangle_adj)

coloring_qubo = graph_3coloring_to_qubo(triangle_adj)

# This problem is larger (9 variables), so let's check if solution is feasible
print(f"\nQUBO problem has {len(coloring_qubo.variable_names)} variables")
print("Variables:", coloring_qubo.variable_names)

if len(coloring_qubo.variable_names) <= 12:  # Manageable size
    opt_x, opt_obj = coloring_qubo.solve_brute_force()
    print("\nOptimal solution found:")
    
    # Interpret coloring
    n_vertices = 3
    colors = ['Red', 'Green', 'Blue']
    vertex_colors = {}
    
    for i in range(n_vertices):
        for c in range(3):
            var_name = f'v{i}_c{c}'
            var_idx = coloring_qubo.variable_names.index(var_name)
            if opt_x[var_idx] == 1:
                vertex_colors[i] = colors[c]
    
    print("Vertex coloring:")
    for vertex, color in vertex_colors.items():
        print(f"  Vertex {vertex}: {color}")
    
    # Verify valid coloring
    valid = True
    for i in range(n_vertices):
        for j in range(i+1, n_vertices):
            if triangle_adj[i, j] == 1:  # Adjacent
                if vertex_colors[i] == vertex_colors[j]:
                    print(f"ERROR: Adjacent vertices {i}, {j} have same color!")
                    valid = False
    
    if valid:
        print("✓ Valid 3-coloring confirmed")
else:
    print("Problem too large for exhaustive search")

print("\n" + "="*50)
print("Tutorial complete! Key takeaways:")
print("1. QUBO formulation enables quantum optimization")
print("2. Penalty methods convert constraints")
print("3. Problem structure affects QUBO complexity")
print("4. Always validate solutions make sense!")